# Évaluation - requêtes SQL

## Principe de fonctionnement

Le principe de fonctionnement est le mêmes pour que pour l'exercice sur les requêtes SQL. Nous ne le répétons pas ici. 


## Configuration

In [1]:
# Un peu de style :
from IPython.display import HTML
HTML("data/style.css")

In [2]:
# On commence par importer le module
from requestdb import RequestDB
# Puis on crée l'objet <db> qui en lui indiquant 
# le chemin vers la base de données (ça fonctionne
# avec n'importe quelle base sqlite)
db = RequestDB("data/films.db")

requestdb v (1, 1)


## Questions

#### Test  :  3 films  de 1998

In [3]:
r = """\
SELECT * FROM film WHERE annee = 1998 LIMIT 3
"""
db.query(r)

id,titre,annee,nbSpectateurs,idRealisateur,idGenre
73,American History X,1998,2984390,814,18
95,Armageddon,1998,69224945,865,28
857,Il faut sauver le soldat Ryan,1998,60230113,488,18


### 1. Requêtes simples

#### 1.1 Liste des genres de films possibles
(il s'agit juste de lister le contenu de la table genre en ne donnant que les intitulés des genres)

In [4]:
# question 1.1
r = """\
SELECT nom AS genres
FROM genre
"""
db.query(r)

genres
Aventure
Fantastique
Animation
Drame
Horreur
Action
Comédie
Histoire
Western
Thriller


#### 1.2 Liste des titres de films commençant par la lettre `R` et sortis en 2010

In [5]:
# question 1.2
r = """\
SELECT titre
FROM film
WHERE titre LIKE 'R%' AND annee=2010
"""
db.query(r)

titre
Remember me
Rien à déclarer


#### 1.3 Liste des noms et dates de naissance des personnes enregistrées dont le prénom commence par `So`...
... du plus âgé au plus jeune (si la date de naissance n'est pas connue, le nom ne doit pas apparaître)

In [6]:
# question 1.3
r = """\
SELECT nom, naissance AS date_naissance
FROM personne
WHERE nom LIKE 'So%' AND naissance IS NOT NULL
ORDER BY naissance
"""
db.query(r)

nom,date_naissance
Sol Gorss,1908-03-22
Sonny Jim Gaines,1928-01-01
Sonny Caldinez,1932-07-01
Sophia Loren,1934-09-20
Souleymane Dicko,1966-08-26
Sophie Marceau,1966-11-17
Sofia Coppola,1971-05-14
Sophie-Marie Larrouy,1984-01-01
Sophie Kennedy Clark,1990-01-01


### 2. Jointures

#### 2.1 Liste des films réalisés par Martin Scorsese, en indiquant à chaque fois leur genre, par ordre alphabétique.

In [7]:
# question 2.1
r = """\
SELECT titre, genre.nom AS genre_de_film
FROM film, genre, personne
WHERE film.idGenre=genre.id AND personne.id=film.idRealisateur AND personne.nom IS 'Martin Scorsese'
-- Un LIKE sur '%Scorsese' aurait pu inclure Fransesca Scorsese qui peut avoir réalisé un film...
ORDER BY titre 
-- je suppose que l'ordre aplphabétique concerne le titre
"""
db.query(r)

titre,genre_de_film
Aviator,Drame
Gangs of New York,Drame
Hugo Cabret,Aventure
Le Loup de Wall Street,Crime
Les Affranchis,Drame
Les Infiltrés,Drame
Shutter Island,Drame


#### 2.2 Liste des titres et nombre d'entrées des films de Quentin Tarantino ayant fait moins de 10 millions d'entrées

In [8]:
# question 2.2
r = """\
SELECT titre, nbSpectateurs AS entrees
FROM film, personne
WHERE personne.id=film.idRealisateur AND personne.nom LIKE 'Quentin%Tarantino%' AND entrees>1E7
-- On peut aussi utiliser -> personne.nom IS 'Quentin Tarantino'
ORDER BY entrees
"""
db.query(r)

titre,entrees
Kill Bill : Volume 2,19019932
Les 8 Salopards,19470014
Kill Bill : Volume 1,22612380
Inglourious Basterds,40181961
Once Upon a Time… in Hollywood,46781405
Django Unchained,53171029


La liste doit s'afficher par ordre croissant du nombre d'entrées.

#### 2.3 Liste des genres de films dans lesquels Léonardo DiCaprio a joué
Il faut qu'il soit acteur et non *juste* réalisateur. Attention à la manière dont le nom de l'acteur est orthographié dans la base. Chaque genre de film ne doit être listé qu'une fois.

In [9]:
# question 2.3
r = """\
SELECT DISTINCT genre.nom AS genre_de_film
FROM film, genre, personne, joue
WHERE film.idGenre=genre.id 
  AND joue.idActeur=personne.id 
  AND film.id=joue.idFilm 
  AND film.IdRealisateur IS NOT personne.id   -- pour exclure son rôle éventuel en tant que réalisateur?
  AND personne.nom IS 'Leonardo DiCaprio'
"""
db.query(r)

genre_de_film
Drame
Romance
Action
Crime
Western
Documentaire


 #### 2.4 Liste des films ayant à l'affiche (en tant qu'acteurs) Léonardo di Caprio et Johnny Depp.

In [10]:
# question 2.4 

r = """\
SELECT DISTINCT titre
FROM film, joue, personne 
WHERE joue.idFilm=film.id
AND joue.idActeur = personne.id AND
personne.nom IN ('Leonardo DiCaprio','Johnny Depp')

GROUP BY film.titre
HAVING COUNT(*) = 2
"""
db.query(r)


titre
Gilbert Grape


### 3. Agrégations


#### 3.1 Année du film le plus ancien qui soit réalisé par un français
Vous vous contenterez de tester : `pays = 'France'`.
Vous n'avez pas le droit d'utiliser `LIMIT`.

In [11]:
# question 3.1
r = """\
SELECT MIN(annee) AS plus_vieux_film_français
FROM film, personne
WHERE personne.id=film.idRealisateur AND personne.pays='France' 
"""
db.query(r)

plus_vieux_film_français
1936


#### 3.2 Liste des pays avec le nombre de films réalisés (par un réalisateur de ce pays), limitée au 10 plus gros producteurs
La liste doit s'afficher par ordre décroissant du nombre de films.

In [12]:
# question 3.2
r = """\
SELECT pays, COUNT(film.id) AS nb_films
FROM film, personne
WHERE film.idRealisateur=personne.id AND pays IS NOT NULL 
GROUP BY pays
ORDER BY nb_films DESC
LIMIT 10
"""
db.query(r)

pays,nb_films
France,372
USA,301
UK,105
Francia,22
New Zealand,16
Germany,12
China,12
Belgium,12
Japan,11
Greece,11


#### 3.3 Liste des réalisateurs avec le nombre de films qu'ils ont réalisé en 2000
Donner la liste par ordre décroissant du nombre de films

In [13]:
# question 3.3
r = """\
SELECT nom, COUNT(film.id) AS nombre
FROM film, personne
WHERE personne.id=film.idRealisateur AND annee=2000
GROUP BY nom
ORDER BY nombre DESC
"""
db.query(r)

nom,nombre
Robert Zemeckis,2
Thaddeus O'Sullivan,1
Ron Howard,1
Rob LaDuca,1
Ridley Scott,1
Mathieu Kassovitz,1
John Woo,1
James Bobin,1
Fabien Onteniente,1
Dominik Moll,1


#### 3.4 Liste des réalisateurs  avec le nombre de films qu'ils ont réalisé entre 2000 et 2010...
Donner la liste par ordre décroissant du nombre de films, et **limitée aux réalisateurs qui ont fait aux moins 4 films**.

In [ ]:
# question 3.4
r = """\
SELECT nom, COUNT(film.id) AS nombre   -- ou COUNT(*)
FROM film, personne
WHERE personne.id=film.idRealisateur AND annee BETWEEN 2000 AND 2010
GROUP BY nom
HAVING nombre>=4    -- ou HAVING COUNT(film.id)>=4
ORDER BY nombre DESC
"""
db.query(r)

#### 3.4 Liste des titres des 10 films ayant le plus gros budget pour les salaires des acteurs (somme des salaires la plus élevée)
Le budget (somme des salaires) doit aussi être donné dans les résultats.

In [ ]:
# question 3.4.bis
r = """\
SELECT titre, SUM(salaire) AS budget_acteurs
FROM film, joue
WHERE joue.idFilm=film.id
GROUP BY titre
ORDER BY budget_acteurs DESC
LIMIT 10
"""
db.query(r)

### Sous-requêtes

#### 4.1 Liste des triplets acteur, film, salaire (acteur ayant joué dans tel film pour tel salaire), tels que le salaire soit supérieur à la moitié du salaire maximum de la base

Il faut que votre requête fonctionne sans modification quelles que soient les
données présentes dans la base.

In [14]:
# question 4.1
r = """\
SELECT nom,titre AS titre_du_film, salaire
FROM film,joue, personne
WHERE joue.idFilm=film.id
    AND joue.idActeur=personne.id
    AND salaire > (SELECT MAX(salaire)/2 FROM joue)
"""
db.query(r)

nom,titre_du_film,salaire
Scarlett Johansson,Avengers : L'Ère d'Ultron,17585120.0
Scarlett Johansson,Thor : Ragnarok,11304720.0
Scarlett Johansson,Avengers: Endgame,22358224.0
Scarlett Johansson,Avengers : Infinity War,18841200.0
Robert Downey Jr.,Avengers : L'Ère d'Ultron,13159440.0
Robert Downey Jr.,Avengers: Endgame,16731288.0
Robert Downey Jr.,Avengers : Infinity War,14099400.0
Henry Cavill,Justice League,14371200.0
Henry Cavill,Batman v Superman : L’aube de la justice,11976000.0
Margot Robbie,Tarzan,11569320.0
